In [1]:
import os

In [2]:
%pwd

'e:\\Trekathon\\Early-Fish-Disease-Detection\\notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Trekathon\\Early-Fish-Disease-Detection'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from early_fish_disease_detection.constants import *
from early_fish_disease_detection.utils.common import read_yaml, create_dir


In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_dir([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_dir([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [8]:
import os
import urllib.request as requests
import zipfile
import gdown
from early_fish_disease_detection import logger
from early_fish_disease_detection.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        "Fetch data from the source URL"

        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix + file_id, zip_download_dir, quiet=False)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
        
        except Exception as e:
            raise e
        
    def unzip_file(self):
        "Unzip the downloaded file"

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_file()
except Exception as e:
    raise e

2025-01-08 12:03:27,575 - Early-Fish-Disease-Detection-logger - INFO - yaml file config\config.yaml read successfully
2025-01-08 12:03:27,963 - Early-Fish-Disease-Detection-logger - INFO - yaml file params.yaml read successfully
2025-01-08 12:03:27,968 - Early-Fish-Disease-Detection-logger - INFO - Directory artifacts created successfully
2025-01-08 12:03:27,974 - Early-Fish-Disease-Detection-logger - INFO - Directory artifacts/data_ingestion created successfully
2025-01-08 12:03:27,981 - Early-Fish-Disease-Detection-logger - INFO - Downloading data from https://drive.google.com/file/d/1SQZ5_wmTlgj5i04qtyMo5BnXbcfq84D-/view?usp=sharing into file artifacts/data_ingestion/data.zip


Downloading...
From: https://drive.google.com/uc?/export=download&id=1SQZ5_wmTlgj5i04qtyMo5BnXbcfq84D-
To: e:\Trekathon\Early-Fish-Disease-Detection\artifacts\data_ingestion\data.zip
100%|██████████| 26.0M/26.0M [00:06<00:00, 4.32MB/s]

2025-01-08 12:03:53,078 - Early-Fish-Disease-Detection-logger - INFO - Downloaded data from https://drive.google.com/file/d/1SQZ5_wmTlgj5i04qtyMo5BnXbcfq84D-/view?usp=sharing into file artifacts/data_ingestion/data.zip
